# Análisis de secuencias de ADN y traducción a proteínas con Biopython

En esta práctica se trabaja con **ficheros FASTA** que contienen secuencias de ADN de varios genes humanos codificantes de proteína.  
El objetivo es:

1. Leer automáticamente varios ficheros FASTA.
2. Calcular el **contenido GC** de cada secuencia.
3. Traducir las secuencias de ADN a **secuencias proteicas**.
4. Guardar las secuencias de proteínas en nuevos ficheros FASTA de salida.

Las proteínas estudiadas son:

- **Insulin (INS)**
- **Hemoglobin beta (HBB)**
- **Myoglobin (MB)**
- **Human serum albumin (ALB)**
- **Keratin (un gen de queratina epitelial, p. ej. KRT8 o similar)**

Estas proteínas tienen funciones clave en fisiología humana:

- La **insulina** es una hormona peptídica producida por las células β pancreáticas que regula el metabolismo de la glucosa, lípidos y proteínas, favoreciendo la captación de glucosa y su almacenamiento como glucógeno o triglicéridos.
- La **hemoglobina** es una proteína tetramérica con grupos hemo que transporta oxígeno desde los pulmones a los tejidos y facilita el retorno de CO₂.
- La **mioglobina** es una proteína monomérica de músculo que almacena oxígeno y facilita su difusión dentro de la fibra muscular, especialmente en músculo esquelético y cardíaco. 
- La **albúmina sérica humana** es la proteína más abundante del plasma; mantiene la presión oncótica y transporta hormonas, ácidos grasos, fármacos y otras moléculas.
- Las **queratinas** de tipo I y II son proteínas de filamentos intermedios que aportan soporte mecánico a las células epiteliales y forman estructuras como piel, cabello y uñas.

Los datos utilizados en esta práctica fueron descargados de las bases de datos públicas del NCBI (National Center for Biotechnology Information)

En las siguientes celdas se muestra paso a paso el código en Python usando **Biopython** y una breve explicación de cada parte.


### 1. Importación de librerías y configuración de rutas


Se utilizan:

- `os` para manejar rutas y carpetas.
- `Bio.SeqIO` para leer y escribir ficheros FASTA.
- `Bio.Seq` y `Bio.SeqRecord` para trabajar con secuencias y registros.

Se asume la siguiente estructura de carpetas:

- `ficheros/` → contiene los FASTA de ADN codificante (`insulin.fna`, `hemoglobin.fna`, etc.).
- `outputs/` → se guardan los FASTA de proteínas generados (`insulin_output.fasta`, etc.).

La ruta base (`BASE_DIR`) debe ajustarse a la ubicación real del proyecto en el sistema.

In [1]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Ruta base del proyecto: ajustar según el sistema
BASE_DIR = r"C:\Users\mariana\OneDrive\Desktop\uni\BIO\practicas\practicaFicheros"

INPUT_DIR = os.path.join(BASE_DIR, "ficheros")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")

# Crear carpeta de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lista de proteínas que se van a analizar
protein_names = [
    "insulin",
    "hemoglobin",
    "myoglobin",
    "albumin",
    "keratin",
]

### 2. Función `leer_fasta`: lectura de ficheros FASTA

Esta función encapsula la lectura de un fichero FASTA utilizando `SeqIO.parse`:

- Entrada: ruta del fichero FASTA.
- Salida: lista de objetos `SeqRecord`, uno por cada secuencia del fichero.

Esto permite reutilizar la misma función para cualquier proteína simplemente cambiando la ruta de entrada.

In [2]:
def leer_fasta(ruta_fasta):
    """
    Lee un archivo FASTA y devuelve una lista de registros (SeqRecord).
    """
    return list(SeqIO.parse(ruta_fasta, "fasta"))


### 3. Función `calcular_gc`: contenido de GC

El **contenido GC** es el porcentaje de nucleótidos que son G o C:

$$
\text{GC\%} = \frac{G + C}{\text{longitud total}} \times 100

$$

La función:

- Convierte la secuencia a mayúsculas para evitar errores.
- Cuenta cuántas `G` y `C` hay.
- Devuelve el porcentaje de GC.
- Si la longitud es 0, devuelve 0 para evitar divisiones por cero.

In [3]:
def calcular_gc(seq):
    """
    Calcula el porcentaje de contenido GC de una secuencia de ADN.
    """
    seq = seq.upper()
    gc = seq.count("G") + seq.count("C")
    return (gc / len(seq)) * 100 if len(seq) > 0 else 0


### 4. Función `traducir_secuencias`: ADN → proteína

Esta función recibe una lista de `SeqRecord` con ADN y:

1. Comprueba si la longitud de la secuencia es múltiplo de 3 (condición necesaria para un marco de lectura completo).
2. Traduce la secuencia a aminoácidos con `seq.translate()`.
3. Crea nuevos `SeqRecord` con la secuencia proteica, manteniendo `id` y `description`.
4. Devuelve la lista de registros de proteínas.

Se usa `to_stop=False` para traducir toda la secuencia, incluyendo el codón de parada que aparecerá como `*` al final si está presente.

In [4]:
def traducir_secuencias(registros):
    """
    Traduce una lista de registros de ADN a proteínas.
    Devuelve una lista de nuevos registros con secuencias proteicas.
    """
    proteinas = []
    for r in registros:
        if len(r.seq) % 3 != 0:
            print(f"  Advertencia: {r.id} tiene longitud no divisible por 3")
        prot = r.seq.translate(to_stop=False)
        prot_record = SeqRecord(prot, id=r.id, description=r.description)
        proteinas.append(prot_record)
    return proteinas


### 5. Función `guardar_fasta`: escritura de FASTA de salida

Una vez traducidas las secuencias, es útil guardarlas en un nuevo fichero FASTA:

- Entrada: lista de `SeqRecord` (proteínas) y ruta de salida.
- Salida: fichero FASTA escrito en disco, más un mensaje informativo.

In [5]:
def guardar_fasta(registros, ruta_salida):
    """
    Guarda una lista de registros en un archivo FASTA.
    """
    SeqIO.write(registros, ruta_salida, "fasta")
    print(f"  Archivo {ruta_salida} generado.")


### 6. Función `pipeline`: ejecución completa para todas las proteínas

La función `pipeline` orquesta todo el proceso:

1. Recorre la lista `protein_names`.
2. Construye la ruta del fichero de entrada (`ficheros/<nombre>.fna`).
3. Comprueba que el fichero existe; si no, muestra un error y pasa a la siguiente proteína.
4. Lee las secuencias con `leer_fasta`.
5. Calcula y muestra el **%GC** de cada secuencia.
6. Traduce las secuencias a proteínas.
7. Guarda las proteínas en `outputs/<nombre>_output.fasta`.

De esta forma, con una sola llamada se procesan de manera automática todas las proteínas elegidas.


In [6]:
def pipeline():
    """
    Ejecuta el análisis completo para todas las proteínas definidas en protein_names.
    """
    for nombre in protein_names:
        print(f"\n=== Procesando {nombre.upper()} ===")

        ruta_entrada = os.path.join(INPUT_DIR, f"{nombre}.fna")
        ruta_salida = os.path.join(OUTPUT_DIR, f"{nombre}_output.fasta")

        if not os.path.exists(ruta_entrada):
            print(f"  [ERROR] No se encuentra el archivo: {ruta_entrada}")
            continue

        # Leer secuencias de ADN
        registros = leer_fasta(ruta_entrada)

        # Calcular y mostrar %GC
        print("  Porcentaje de GC por secuencia:")
        for r in registros:
            gc = calcular_gc(str(r.seq))
            print(f"    {r.id} -> GC%: {gc:.2f}")

        # Traducir a proteínas y guardar resultados
        proteinas_traducidas = traducir_secuencias(registros)
        guardar_fasta(proteinas_traducidas, ruta_salida)


### 7. Ejecución del pipeline

Al ejecutar la siguiente celda, el notebook:

- Leerá los ficheros `insulin.fna`, `hemoglobin.fna`, `myoglobin.fna`, `albumin.fna` y `keratin.fna` desde la carpeta `ficheros`.
- Mostrará por pantalla el porcentaje de GC de cada secuencia.
- Creará los ficheros de salida con las proteínas traducidas en la carpeta `outputs`.

In [7]:
pipeline()


=== Procesando INSULIN ===
  Porcentaje de GC por secuencia:
    NM_000207.3:60-392 -> GC%: 64.56
    NM_001185097.2:86-418 -> GC%: 64.56
    NM_001185098.2:239-571 -> GC%: 64.56
    NM_001291897.2:120-452 -> GC%: 64.56
  Archivo C:\Users\mariana\OneDrive\Desktop\uni\BIO\practicas\practicaFicheros\outputs\insulin_output.fasta generado.

=== Procesando HEMOGLOBIN ===
  Porcentaje de GC por secuencia:
    NM_000518.5:51-494 -> GC%: 56.08
  Archivo C:\Users\mariana\OneDrive\Desktop\uni\BIO\practicas\practicaFicheros\outputs\hemoglobin_output.fasta generado.

=== Procesando MYOGLOBIN ===
  Porcentaje de GC por secuencia:
    NM_001362846.2:61-525 -> GC%: 57.20
    NM_001382809.1:289-753 -> GC%: 57.20
    NM_001382810.1:177-641 -> GC%: 57.20
    NM_001382811.1:246-710 -> GC%: 57.20
    NM_001382812.1:123-422 -> GC%: 57.33
    NM_001382813.1:235-534 -> GC%: 57.33
    NM_005368.3:73-537 -> GC%: 57.20
    NM_203377.1:173-637 -> GC%: 57.20
    NM_203378.1:156-620 -> GC%: 57.20
  Archivo C:\Use

### 9. Conclusiones

En esta práctica se ha desarrollado un flujo de trabajo completo para el análisis automático de secuencias de ADN utilizando Biopython. El pipeline implementado permite leer múltiples ficheros FASTA, calcular el porcentaje de GC, traducir las secuencias a proteínas y generar archivos FASTA de salida con los resultados.

Los análisis realizados sobre cinco proteínas humanas (insulin, hemoglobin, myoglobin, albumin y keratin) muestran diferencias biológicamente coherentes en la composición nucleotídica de sus regiones codificantes (CDS):

- **Insulin** presenta un contenido GC muy alto (**64.56 %** en las cuatro isoformas analizadas). Esto concuerda con que el gen INS es corto, altamente conservado y con fuerte expresión en tejido pancreático.
- **Hemoglobin β** muestra un GC del **56.08 %**, característico de genes muy expresados en eritropoyesis.
- **Myoglobin** mantiene valores relativamente homogéneos entre sus distintos transcritos, con un contenido GC en torno al **57.20–57.33 %**, reflejando su función especializada en músculo.
- **Albumin**, por el contrario, presenta un valor claramente más bajo (**42.95 %**), consistente con su mayor longitud, estructura más compleja y composición nucleotídica más equilibrada.
- **Keratin** muestra un GC medio-alto (**54.63 %**), coherente con la familia de queratinas, cuyos genes suelen ser extensos y ricos en GC en ciertas regiones estructurales.

Además, para todas las proteínas estudiadas se generaron correctamente archivos FASTA con las secuencias proteicas traducidas, verificando la integridad de las CDS (longitudes múltiplos de tres y traducciones completas que incluyen codón de parada).

En conjunto, esta práctica demuestra cómo integrar herramientas bioinformáticas para automatizar procesos recurrentes en el análisis de secuencias, permitiendo realizar estudios comparativos de manera rápida, reproducible y escalable.```

